In [37]:
import cryptpandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = cryptpandas.read_encrypted(path="./release_4827.crypt", password='KZpaNLmv4lDHBN6O')

In [38]:
data

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_32,strat_33,strat_34,strat_35,strat_36,strat_37,strat_38,strat_39,strat_40,strat_41
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.045104,NaN,NaN,NaN,NaN,10.139780,1.481773,NaN,1.298774,NaN,...,NaN,-0.424850,0.069012,-0.799573,NaN,NaN,-1.304132,NaN,NaN,NaN
2,-0.229267,NaN,NaN,NaN,NaN,10.109780,0.731916,NaN,1.748748,NaN,...,NaN,-0.221692,0.709644,-0.463345,NaN,NaN,1.804351,NaN,NaN,NaN
3,-0.332475,NaN,-0.056052,NaN,NaN,9.961956,-0.557418,-0.246940,0.043789,NaN,...,NaN,1.383381,-0.253581,1.512385,NaN,NaN,-1.566135,NaN,NaN,NaN
4,-0.827446,NaN,-0.349031,NaN,NaN,9.964834,-1.177777,-0.795178,-1.105112,NaN,...,NaN,0.550712,-0.145675,0.716065,NaN,NaN,0.309988,NaN,-0.978291,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4823,-0.050160,-0.499776,-0.419339,-0.889985,10.114493,10.972194,-0.530065,0.583815,1.201068,-1.968037,...,-1.152579,0.182387,-0.164327,0.376054,-0.496815,1.120386,0.759048,-0.934667,1.793972,-0.786565
4824,-0.015187,-0.116524,1.236154,1.326128,10.812942,9.772223,0.618545,0.235038,-0.316426,1.525948,...,-0.043252,-1.201451,1.235924,-1.226394,-0.479974,-0.026450,0.030198,0.247484,-0.462805,1.190727
4825,0.065203,-0.145534,1.510954,1.109522,10.008542,9.870874,3.130866,1.422568,-0.164230,0.896664,...,0.348672,-0.106458,0.205386,0.128205,0.457078,0.370953,0.554079,0.259574,0.306446,-0.158908
4826,-2.043694,0.038409,1.674341,1.693272,10.474907,8.500921,1.956392,-0.137554,-1.910148,3.150741,...,-0.204593,-1.518430,1.249149,-2.251074,2.060513,0.050055,-0.475493,-1.687587,-2.989358,0.758773


In [39]:
def clean_data(data):
    """
    Clean the dataset by handling NaN values and infinite values
    
    Parameters:
    data (pd.DataFrame): Raw DataFrame containing strategy returns
    
    Returns:
    pd.DataFrame: Cleaned DataFrame
    """
    # Remove rows where all values are NaN
    cleaned_data = data.dropna(how='all')
    
    # Forward fill remaining NaN values
    cleaned_data = cleaned_data.fillna(method='ffill')
    
    # Remove infinite values (replace with NaN first, then forward fill)
    cleaned_data = cleaned_data.replace([np.inf, -np.inf], np.nan)
    cleaned_data = cleaned_data.fillna(method='ffill')
    
    return cleaned_data

In [40]:
def calculate_sharpe_ratios(returns_df, risk_free_rate=0.02):
    """
    Calculate Sharpe ratios for multiple trading strategies
    
    Parameters:
    returns_df (pd.DataFrame): DataFrame containing returns for each strategy
    risk_free_rate (float): Annual risk-free rate (default 0.02 or 2%)
    
    Returns:
    pd.Series: Sharpe ratios for each strategy
    """
    # Convert annual risk-free rate to daily
    rf_daily = (1 + risk_free_rate) ** (1/252) - 1
    
    # Calculate mean returns for each strategy
    mean_returns = returns_df.mean()
    
    # Calculate volatility using log returns
    log_returns = np.log(1 + returns_df)
    volatility = log_returns.std() * np.sqrt(len(returns_df))
    
    # Calculate Sharpe ratio
    sharpe_ratios = (mean_returns - rf_daily) / volatility
    
    return sharpe_ratios



In [43]:
def plot_sharpe_ratios(sharpe_ratios):
    """
    Create a bar plot of Sharpe ratios
    
    Parameters:
    sharpe_ratios (pd.Series): Series containing Sharpe ratios for each strategy
    """
    plt.figure(figsize=(15, 6))
    sorted_sharpe = sharpe_ratios.sort_values(ascending=False)
    sorted_sharpe.plot(kind='bar')
    plt.title('Sharpe Ratios by Strategy')
    plt.xlabel('Strategy')
    plt.ylabel('Sharpe Ratio')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Example usage:
# Clean the data
cleaned_data = clean_data(data)

# Calculate Sharpe ratios
sharpe_ratios = calculate_sharpe_ratios(cleaned_data)

# Sort and display results
sorted_sharpe = sharpe_ratios.sort_values(ascending=False)
print("Sharpe Ratios for each strategy (sorted):")
print(sorted_sharpe)


Sharpe Ratios for each strategy (sorted):
strat_4        1.498197
strat_5        1.359819
strat_38       0.002015
strat_25       0.002013
strat_31       0.001761
strat_21       0.001697
strat_37       0.001569
strat_30       0.001316
strat_34       0.001221
strat_26       0.001069
strat_29       0.001039
strat_19       0.000917
strat_41       0.000846
strat_7        0.000843
strat_3        0.000806
strat_8        0.000798
strat_32       0.000788
strat_24       0.000701
strat_40       0.000684
strat_12       0.000526
strat_27       0.000458
strat_6        0.000427
strat_17       0.000404
strat_22       0.000315
strat_0        0.000289
strat_15       0.000256
strat_35       0.000227
strat_16       0.000197
strat_28       0.000193
strat_23       0.000053
strat_2        0.000033
strat_11      -0.000035
strat_36      -0.000258
strat_18      -0.000270
strat_20      -0.000346
strat_39      -0.000365
strat_33      -0.000653
strat_13      -0.000723
strat_10      -0.001082
strat_1       -0.00171

c:\Users\alw\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [42]:
def greedy_allocate(arr):
    ranked = list(sorted(arr, key=abs, reverse=True))[:10]
    weights = [0] * len(arr)
    for i, n in enumerate(arr):
        if n in ranked:
            weights[i] = 0.1 if n > 0 else -0.1
    return weights

def convert_to_submission(weights):
    res = dict()
    for i in range(len(weights)):
        res[f"strat_{i}"] = weights[i]
    res['team_name'] = 'Yoghurt'
    res['passcode'] = 'yoghurt'
    return res

convert_to_submission(greedy_allocate(sharpe_ratios))


{'strat_0': 0,
 'strat_1': -0.1,
 'strat_2': 0,
 'strat_3': 0,
 'strat_4': 0.1,
 'strat_5': 0.1,
 'strat_6': 0,
 'strat_7': 0,
 'strat_8': 0,
 'strat_9': -0.1,
 'strat_10': 0,
 'strat_11': 0,
 'strat_12': 0,
 'strat_13': 0,
 'strat_14': -0.1,
 'strat_15': 0,
 'strat_16': 0,
 'strat_17': 0,
 'strat_18': 0,
 'strat_19': 0,
 'strat_20': 0,
 'strat_21': 0.1,
 'strat_22': 0,
 'strat_23': 0,
 'strat_24': 0,
 'strat_25': 0.1,
 'strat_26': 0,
 'strat_27': 0,
 'strat_28': 0,
 'strat_29': 0,
 'strat_30': 0,
 'strat_31': 0.1,
 'strat_32': 0,
 'strat_33': 0,
 'strat_34': 0,
 'strat_35': 0,
 'strat_36': 0,
 'strat_37': 0.1,
 'strat_38': 0.1,
 'strat_39': 0,
 'strat_40': 0,
 'strat_41': 0,
 'team_name': 'Yoghurt',
 'passcode': 'yoghurt'}